In [1]:
import pandas as pd
import numpy as np

In [2]:
base_nba_per_game = pd.read_csv("bases_nba_stats/base_nba_per_game_17_18.csv")

In [3]:
base_nba_per_game.head()

,GAME,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,FGM,FGA,FG_PCT,...,CONTESTED_SHOTS_2PT,CONTESTED_SHOTS_3PT,DEFLECTIONS,LOOSE_BALLS_RECOVERED,CHARGES_DRAWN,SCREEN_ASSISTS,BOX_OUTS,PTS_hustle,GAME_DATE,GAME_PLACE
0,BOS @ CLE 2017-10-17,21700001,1610612739,Cavaliers,CLE,Cleveland,240:00,38,83,0.458,...,44,16,17,14,2,9,38,99,2017-10-17,Quicken Loans Arena
1,BOS @ CLE 2017-10-17,21700001,1610612738,Celtics,BOS,Boston,240:00,36,88,0.409,...,32,22,11,1,1,5,32,102,2017-10-17,Quicken Loans Arena
2,HOU @ GSW 2017-10-17,21700002,1610612745,Rockets,HOU,Houston,240:00,47,97,0.485,...,30,26,23,10,2,7,21,122,2017-10-17,ORACLE Arena
3,HOU @ GSW 2017-10-17,21700002,1610612744,Warriors,GSW,Golden State,240:00,43,80,0.537,...,46,35,9,11,0,6,27,121,2017-10-17,ORACLE Arena
4,CHA @ DET 2017-10-18,21700003,1610612765,Pistons,DET,Detroit,240:00,41,96,0.427,...,45,19,6,8,0,3,16,90,2017-10-18,Little Caesars Arena
